In [1]:
!pip install gdown

In [2]:
import gdown
import zipfile

# File ID and URL
file_id = "1Kf6dCn1u5Qu9ijt0yltd9tzXI7Dd06W0"
url = "https://drive.google.com/file/d/1f5pAbU5OdnxMPZxIebUFvJeroOXxbWEZ/view?usp=drive_link"
output = "sketches_png.zip"

# Download zip file
gdown.download(url, output, fuzzy=True)

# Extract zip into 'outlier_DATA' folder
with zipfile.ZipFile(output, "r") as zip_ref:
    zip_ref.extractall("sketches")




Downloading...
From (original): https://drive.google.com/uc?id=1f5pAbU5OdnxMPZxIebUFvJeroOXxbWEZ
From (redirected): https://drive.google.com/uc?id=1f5pAbU5OdnxMPZxIebUFvJeroOXxbWEZ&confirm=t&uuid=030448bd-9703-4030-90c4-d3e54ea68b6b
To: /content/sketches_png.zip
100%|██████████| 550M/550M [00:10<00:00, 53.2MB/s]


In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define root directory
root_dir = "sketches/png"  # Root folder containing class-wise subfolders
output_size = (128, 128)  # Resize dimensions

# Initialize dataset and labels
X = []
y = []
class_labels = {}

# Iterate through each class folder
for label_idx, class_name in enumerate(os.listdir(root_dir)):
    class_path = os.path.join(root_dir, class_name)

    if not os.path.isdir(class_path):  # Skip non-directory files
        continue

    class_labels[label_idx] = class_name  # Store label index mapping

    for img_file in os.listdir(class_path):
        if img_file.endswith(".png"):  # Only process PNG images
            img_path = os.path.join(class_path, img_file)

            # Read image in grayscale
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Resize to 128x128
            img_resized = cv2.resize(img, output_size)

            # Normalize pixel values (0 to 1)
            img_normalized = img_resized / 255.0

            X.append(img_normalized)
            y.append(label_idx)  # Assign label

# Convert lists to NumPy arrays
X = np.array(X)
y = np.array(y)

# Split into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dataset prepared: {X_train.shape[0]} training images, {X_test.shape[0]} testing images.")
print(f"Class labels mapping: {class_labels}")


Dataset prepared: 16000 training images, 4000 testing images.
Class labels mapping: {0: 'radio', 1: 'pretzel', 2: 'wheelbarrow', 3: 'mailbox', 4: 'purse', 5: 'tooth', 6: 'ipod', 7: 'toilet', 8: 'satellite dish', 9: 'parrot', 10: 'sheep', 11: 'frog', 12: 'stapler', 13: 'shovel', 14: 'space shuttle', 15: 'floor lamp', 16: 'hammer', 17: 'beer-mug', 18: 'angel', 19: 'backpack', 20: 'bush', 21: 'truck', 22: 'motorbike', 23: 'butterfly', 24: 'rainbow', 25: 'basket', 26: 'arm', 27: 'tomato', 28: 'vase', 29: 'tree', 30: 'dog', 31: 'door', 32: 'cigarette', 33: 'bench', 34: 'flashlight', 35: 'squirrel', 36: 'trousers', 37: 'hat', 38: 'giraffe', 39: 'head', 40: 'train', 41: 'wrist-watch', 42: 'donut', 43: 'head-phones', 44: 'potted plant', 45: 'fork', 46: 'elephant', 47: 'spider', 48: 'bottle opener', 49: 'candle', 50: 'ant', 51: 'binoculars', 52: 'dolphin', 53: 'hot air balloon', 54: 'bed', 55: 'sword', 56: 'windmill', 57: 'helmet', 58: 'pipe (for smoking)', 59: 'violin', 60: 'rifle', 61: 'spong

In [6]:
import cv2
import numpy as np

def enhance_contrast(image):
    """Apply Histogram Equalization to enhance contrast."""
    return cv2.equalizeHist((image * 255).astype(np.uint8))  # Convert back to 8-bit

# Apply contrast enhancement to training and testing images
contrast_X_train = np.array([enhance_contrast(img) for img in X_train])
contrast_X_test = np.array([enhance_contrast(img) for img in X_test])

# Normalize again
contrast_X_train = contrast_X_train / 255.0
contrast_X_test = contrast_X_test / 255.0


In [7]:
def denoise_image(image):
    """Apply Median Blur to remove noise while preserving edges."""
    return cv2.medianBlur((image * 255).astype(np.uint8), 3)  # Convert back to 8-bit before blurring

# Apply denoising to training and testing images
denoised_X_train = np.array([denoise_image(img) for img in contrast_X_train])
denoised_X_test = np.array([denoise_image(img) for img in contrast_X_test])

# Normalize again
denoised_X_train = denoised_X_train / 255.0
denoised_X_test = denoised_X_test / 255.0

